In [2]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [3]:
subprocess.run([
    'python', 'train_scripts/test_1110.py', 
    # '-s', 'saved/1111_Gosai_MPRA_MyResTrans_seq_AG_VEF/1110_135647/', 
    # '-s', 'saved/1111_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1110_135744/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_AG_VEF_finetune/1112_014010/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1112_013905/', 
    # '-s', 'saved/1113_Gosai_MPRA_MyResTransLateFusion_seq_Sei_VEF_finetune/1112_071608/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_AG_VDNase_finetune/1112_071248/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VDNase_finetune/1112_071330/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VDNase/1112_114300/', 
    # '-s', 'saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_164443/', 
    '-s', 'saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_173424/', 
    '-o', 'Gosai_pred.npy',
    '-de', 'cuda:0'],
    cwd='..'
)

use saved config: saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_173424/config.yaml


100%|██████████| 186/186 [00:33<00:00,  5.47it/s]


CompletedProcess(args=['python', 'train_scripts/test_1110.py', '-s', 'saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_173424/', '-o', 'Gosai_pred.npy', '-de', 'cuda:0'], returncode=0)

In [4]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_760679.tsv', sep='\t')
MPRA_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,HCT116,A549
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.018,-0.048,0.017,-0.708,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-1.043,0.191,0.099,-0.262,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-1.005,0.650,-0.185,0.627,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.779,0.762,0.281,0.676,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.455,-0.643,0.066,-0.164,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,0.716,1.309,1.089,NaN,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.708,-0.544,-0.427,NaN,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,-0.143,-0.195,-0.308,NaN,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,-0.116,-0.084,-0.385,NaN,NaN


In [5]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

In [6]:
data_split_dict = {}

data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)
data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(3, 23)) + ['X', 'Y'])])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test']  = MPRA_df['chr'].isin(['chr2'])

# 和前三种细胞类型差异top5%的序列定义为cell type specific
for cell_type in cell_types:
    ref_mean = MPRA_df[cell_types[:3]].mean(axis=1)
    diff = (MPRA_df[cell_type] - ref_mean).abs()
    threshold = np.percentile(diff.dropna(), 99)
    data_split_dict[f'{cell_type}_specific'] = diff > threshold


for key in data_split_dict:
    print(key, data_split_dict[key].sum())

keys = list(data_split_dict.keys())
for k1 in keys:
    for k2 in keys:
        data_split_dict[f'{k1}+{k2}'] = data_split_dict[k1] & data_split_dict[k2]

total 760679
train 631073
valid 70952
test 58654
K562_specific 7607
HepG2_specific 7607
SK-N-SH_specific 7607
HCT116_specific 4482
A549_specific 3195


In [7]:
# y_pred = np.load(f'../saved/1110_Gosai_MPRA_MyResTransLateFusion_seq_Sei_VEF/1110_120414/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1110_Gosai_MPRA_MyResTransLateFusion_seq_Sei_VEF_cls/1111_014737/Gosai_pred.npy')

# y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_AG_VEF/1110_135647/Gosai_pred.npy') # train更好，unseen差不多，就用这个
# 864, 852, 843, 815, 705
# 804, 755, 779, 687, 731
# y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1110_135744/Gosai_pred.npy')# Sei 版本
# y_pred = np.load(f'../saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1112_013905/Gosai_pred.npy') # finetune版本,unseen上效果变差
# y_pred = np.load(f'../saved/1112_Gosai_MPRA_MyResTrans_seq_AG_VEF_finetune/1112_014010/Gosai_pred.npy') # finetune版本,unseen上效果变差
# y_pred = np.load(f'../saved/1113_Gosai_MPRA_MyResTransLateFusion_seq_Sei_VEF_finetune/1112_071608/Gosai_pred.npy')

# y_pred = np.load(f'../saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VDNase_finetune/1112_071330/Gosai_pred.npy')

# y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1110_135744/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_AG_VEF/1110_135647/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VDNase/1112_114300/Gosai_pred.npy')


# y_pred = np.load(f'../saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_164443/Gosai_pred.npy')
y_pred = np.load(f'../saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_173424/Gosai_pred.npy')

pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[pred_cols] = y_pred
MPRA_df.describe()

,pos,K562,HepG2,SK-N-SH,HCT116,A549,K562_pred,HepG2_pred,SK-N-SH_pred,HCT116_pred,A549_pred
count,760679.000,760679.000,760679.000,760679.000,448103.000,319496.000,760679.000,760679.000,760679.000,760679.000,760679.000
mean,75202958.598,-0.000,-0.000,0.000,0.000,-0.000,0.112,0.097,0.094,0.100,0.100
std,57521411.054,1.000,1.000,1.000,1.000,1.000,1.059,1.012,1.013,1.034,1.024
min,14677.000,-5.360,-5.745,-5.762,-5.956,-6.017,-4.601,-4.659,-4.657,-4.624,-4.622
25%,28973267.000,-0.564,-0.597,-0.569,-0.567,-0.504,-0.465,-0.456,-0.461,-0.460,-0.458
50%,62363640.000,-0.283,-0.256,-0.206,-0.272,-0.203,-0.220,-0.218,-0.215,-0.222,-0.217
75%,112600466.000,0.206,0.284,0.285,0.208,0.215,0.255,0.243,0.243,0.234,0.243
max,249200985.000,7.854,7.705,8.315,7.761,11.771,8.081,7.962,8.028,8.082,8.051


In [8]:
for data_split in ['total', 'test', 'test+specific']:
    pearson_df = pd.DataFrame()
    for c1 in cell_types:
        for c2 in cell_types:
            if data_split == 'specific':
                condition = data_split_dict[f'{c1}_specific']
            elif data_split == 'test+specific':
                condition = data_split_dict[f'{c1}_specific'] & data_split_dict['test']
            else:
                condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{c1}']
            y = df[f'{c2}_pred']
            r, p = pearson(x, y)
            pearson_df.loc[f'{c1}', f'{c2}_pred'] = r
    print(data_split, len(x))
    print(pearson_df)

total 760679
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.959       0.938         0.931        0.940      0.941
HepG2        0.789       0.804         0.784        0.792      0.795
SK-N-SH      0.749       0.746         0.747        0.748      0.752
HCT116       0.823       0.831         0.819        0.835      0.830
A549         0.709       0.713         0.694        0.727      0.729
test 58654
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.858       0.830         0.821        0.836      0.838
HepG2        0.729       0.743         0.717        0.733      0.739
SK-N-SH      0.695       0.686         0.688        0.692      0.697
HCT116       0.771       0.777         0.763        0.787      0.780
A549         0.642       0.645         0.623        0.661      0.665
test+specific 281
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.750       0.616         0.649        0.638    

In [9]:
y_pred = np.load(f'../saved/1110_Gosai_MPRA_MyResTrans_seq/1110_055401/Gosai_pred.npy')

pred_cols = [f'{cell_type}_pred' for cell_type in cell_types[:3]]
MPRA_df[pred_cols] = y_pred
MPRA_df['HCT116_pred'] = y_pred.mean(axis=1)
MPRA_df['A549_pred'] = y_pred.mean(axis=1)
MPRA_df.describe()

,pos,K562,HepG2,SK-N-SH,HCT116,A549,K562_pred,HepG2_pred,SK-N-SH_pred,HCT116_pred,A549_pred
count,760679.000,760679.000,760679.000,760679.000,448103.000,319496.000,760679.000,760679.000,760679.000,760679.000,760679.000
mean,75202958.598,-0.000,-0.000,0.000,0.000,-0.000,0.111,0.092,0.075,0.093,0.093
std,57521411.054,1.000,1.000,1.000,1.000,1.000,1.084,1.063,1.056,1.016,1.016
min,14677.000,-5.360,-5.745,-5.762,-5.956,-6.017,-1.636,-3.053,-2.748,-2.216,-2.216
25%,28973267.000,-0.564,-0.597,-0.569,-0.567,-0.504,-0.502,-0.547,-0.528,-0.501,-0.501
50%,62363640.000,-0.283,-0.256,-0.206,-0.272,-0.203,-0.267,-0.237,-0.191,-0.222,-0.222
75%,112600466.000,0.206,0.284,0.285,0.208,0.215,0.279,0.358,0.320,0.317,0.317
max,249200985.000,7.854,7.705,8.315,7.761,11.771,7.747,7.648,8.331,7.779,7.779


In [10]:
y_pred = np.load(f'../saved/1110_Gosai_MPRA_MyResTrans_seq5/1112_093253/Gosai_pred.npy')

pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[pred_cols] = y_pred
MPRA_df.describe()

,pos,K562,HepG2,SK-N-SH,HCT116,A549,K562_pred,HepG2_pred,SK-N-SH_pred,HCT116_pred,A549_pred
count,760679.000,760679.000,760679.000,760679.000,448103.000,319496.000,760679.000,760679.000,760679.000,760679.000,760679.000
mean,75202958.598,-0.000,-0.000,0.000,0.000,-0.000,0.091,0.073,0.080,0.002,0.074
std,57521411.054,1.000,1.000,1.000,1.000,1.000,1.063,1.041,1.037,0.981,1.216
min,14677.000,-5.360,-5.745,-5.762,-5.956,-6.017,-1.647,-3.726,-2.913,-2.860,-1.935
25%,28973267.000,-0.564,-0.597,-0.569,-0.567,-0.504,-0.512,-0.559,-0.524,-0.551,-0.533
50%,62363640.000,-0.283,-0.256,-0.206,-0.272,-0.203,-0.278,-0.247,-0.184,-0.311,-0.278
75%,112600466.000,0.206,0.284,0.285,0.208,0.215,0.255,0.332,0.338,0.142,0.164
max,249200985.000,7.854,7.705,8.315,7.761,11.771,7.559,7.564,8.159,8.266,11.658


In [11]:
for data_split in ['total', 'test', 'test+specific']:
    pearson_df = pd.DataFrame()
    for c1 in cell_types:
        for c2 in cell_types:
            if data_split == 'specific':
                condition = data_split_dict[f'{c1}_specific']
            elif data_split == 'test+specific':
                condition = data_split_dict[f'{c1}_specific'] & data_split_dict['test']
            else:
                condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{c1}']
            y = df[f'{c2}_pred']
            r, p = pearson(x, y)
            pearson_df.loc[f'{c1}', f'{c2}_pred'] = r
    print(data_split, len(x))
    print(pearson_df)

total 760679
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.943       0.813         0.780        0.829      0.809
HepG2        0.814       0.946         0.852        0.867      0.816
SK-N-SH      0.772       0.846         0.937        0.828      0.854
HCT116       0.845       0.875         0.846        0.947      0.854
A549         0.729       0.756         0.802        0.773      0.909
test 58654
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.855       0.747         0.717        0.755      0.740
HepG2        0.735       0.854         0.772        0.778      0.732
SK-N-SH      0.698       0.765         0.848        0.735      0.772
HCT116       0.775       0.802         0.770        0.864      0.776
A549         0.642       0.673         0.697        0.665      0.793
test+specific 281
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.722       0.126         0.056        0.245    

In [12]:
# VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_AG_VEF_scalelog1p.tsv', sep='\t')
VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Sei_VEF_zscore.tsv', sep='\t')
# VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Enformer_VEF_log1p.tsv', sep='\t')
# VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Borzoi_VEF_log1p.tsv', sep='\t')


DNase_cols = [f'{cell_type}_DNase' for cell_type in cell_types]

pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[pred_cols] = VEF_df[DNase_cols]
MPRA_df.describe()

,pos,K562,HepG2,SK-N-SH,HCT116,A549,K562_pred,HepG2_pred,SK-N-SH_pred,HCT116_pred,A549_pred
count,760679.000,760679.000,760679.000,760679.000,448103.000,319496.000,760679.000,760679.000,760679.000,760679.000,760679.000
mean,75202958.598,-0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000
std,57521411.054,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,14677.000,-5.360,-5.745,-5.762,-5.956,-6.017,-3.071,-2.539,-3.506,-3.177,-3.474
25%,28973267.000,-0.564,-0.597,-0.569,-0.567,-0.504,-0.706,-0.734,-0.682,-0.687,-0.694
50%,62363640.000,-0.283,-0.256,-0.206,-0.272,-0.203,-0.202,-0.210,-0.233,-0.191,-0.145
75%,112600466.000,0.206,0.284,0.285,0.208,0.215,0.493,0.543,0.483,0.456,0.497
max,249200985.000,7.854,7.705,8.315,7.761,11.771,6.455,5.766,5.613,7.113,6.630


In [13]:
for data_split in ['total', 'test', 'test+specific']:
    pearson_df = pd.DataFrame()
    for c1 in cell_types:
        for c2 in cell_types:
            if data_split == 'specific':
                condition = data_split_dict[f'{c1}_specific']
            elif data_split == 'test+specific':
                condition = data_split_dict[f'{c1}_specific'] & data_split_dict['test']
            else:
                condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{c1}']
            y = df[f'{c2}_pred']
            r, p = pearson(x, y)
            pearson_df.loc[f'{c1}', f'{c2}_pred'] = r
    print(data_split, len(x))
    print(pearson_df)

total 760679
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.535       0.400         0.410        0.445      0.427
HepG2        0.522       0.483         0.451        0.482      0.476
SK-N-SH      0.469       0.384         0.420        0.420      0.415
HCT116       0.492       0.431         0.437        0.483      0.454
A549         0.362       0.302         0.280        0.363      0.347
test 58654
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.508       0.381         0.388        0.419      0.404
HepG2        0.484       0.458         0.417        0.446      0.447
SK-N-SH      0.433       0.351         0.388        0.383      0.379
HCT116       0.465       0.410         0.413        0.452      0.425
A549         0.323       0.275         0.254        0.328      0.317
test+specific 281
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.586       0.148         0.217        0.205    

In [6]:
# y_pred = np.load(f'Gosai_MPRA_Sei_VEF_linear_pred.npy')
y_pred = np.load(f'Gosai_MPRA_Sei_VEF_MLP_pred.npy')

pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[pred_cols] = y_pred
MPRA_df.describe()

,pos,K562,HepG2,SK-N-SH,HCT116,A549,K562_pred,HepG2_pred,SK-N-SH_pred,HCT116_pred,A549_pred
count,760679.000,760679.000,760679.000,760679.000,448103.000,319496.000,760679.000,760679.000,760679.000,760679.000,760679.000
mean,75202958.598,-0.000,-0.000,0.000,0.000,-0.000,-0.010,-0.009,0.032,0.001,0.009
std,57521411.054,1.000,1.000,1.000,1.000,1.000,0.610,0.576,0.631,0.630,0.623
min,14677.000,-5.360,-5.745,-5.762,-5.956,-6.017,-3.586,-1.045,-3.354,-2.111,-1.487
25%,28973267.000,-0.564,-0.597,-0.569,-0.567,-0.504,-0.309,-0.308,-0.289,-0.321,-0.305
50%,62363640.000,-0.283,-0.256,-0.206,-0.272,-0.203,-0.214,-0.211,-0.162,-0.222,-0.200
75%,112600466.000,0.206,0.284,0.285,0.208,0.215,0.021,0.036,0.078,0.038,0.037
max,249200985.000,7.854,7.705,8.315,7.761,11.771,6.708,6.176,6.740,6.214,6.601


In [7]:
for data_split in ['total', 'test', 'test+specific']:
    pearson_df = pd.DataFrame()
    for c1 in cell_types:
        for c2 in cell_types:
            if data_split == 'specific':
                condition = data_split_dict[f'{c1}_specific']
            elif data_split == 'test+specific':
                condition = data_split_dict[f'{c1}_specific'] & data_split_dict['test']
            else:
                condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{c1}']
            y = df[f'{c2}_pred']
            r, p = pearson(x, y)
            pearson_df.loc[f'{c1}', f'{c2}_pred'] = r
    print(data_split, len(x))
    print(pearson_df)

total 760679
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.634       0.522         0.520        0.568      0.536
HepG2        0.593       0.590         0.547        0.597      0.576
SK-N-SH      0.580       0.530         0.581        0.579      0.553
HCT116       0.596       0.565         0.550        0.618      0.578
A549         0.468       0.434         0.431        0.499      0.489
test 58654
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.601       0.488         0.490        0.538      0.505
HepG2        0.559       0.560         0.515        0.564      0.548
SK-N-SH      0.554       0.495         0.555        0.546      0.520
HCT116       0.570       0.533         0.522        0.583      0.549
A549         0.424       0.398         0.405        0.461      0.451
test+specific 281
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.507       0.022         0.049        0.149    

In [9]:
# for data_split in ['total', 'test', 'specific', 'test+specific']:
#     pearson_df = pd.DataFrame()
#     for c1 in cell_types:
#         for c2 in cell_types:
#             if data_split == 'specific':
#                 condition = data_split_dict[f'{c1}_specific']
#             elif data_split == 'test+specific':
#                 condition = data_split_dict[f'{c1}_specific'] & data_split_dict['test']
#             else:
#                 condition = data_split_dict[data_split]
#             df = MPRA_df[condition]
#             # x = df[f'{c1}']
#             # y = df[f'{c2}_pred']
            
#             mean = df[cell_types[:3]].mean(axis=1)
#             mean_pred = df[pred_cols[:3]].mean(axis=1)
#             x = df[f'{c1}'] - mean
#             y = df[f'{c2}_pred'] - mean_pred
#             r, p = pearson(x, y)
#             pearson_df.loc[f'{c1}', f'{c2}_pred'] = r
#     print(data_split, len(x))
#     print(pearson_df)